In [1]:
import os
import time
import json
import pickle
import timeit
import random
import param
import shutil
import collections
import numpy as np
import tensorflow as tf

import stctrain
import datahelper
import stctokenizer
import nuggetdetection as ND
import dialogquality as DQ
import dialogquality_ndfeature as DQNDF
import stcevaluation as STCE

from scipy import stats
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

import logging
logging.basicConfig(level=logging.ERROR)

Using TensorFlow backend.


In [2]:
doclen = param.doclen
embsize = param.embsize
max_sent = param.max_sent
NDclasses = param.NDclasses
DQclasses = param.DQclasses
sentembsize = param.sentembsize

REMOVE_STOPWORDS = False
TO_LOWER = True
TOKEN_TYPE = 'nltk'
EMB = 'stc' # glove or stc

datahelper = datahelper.DataHelper(embedding_path="../embedding/STCWiki/STCWiki_mincount0.model.bin")
stctokenizer = stctokenizer.STCTokenizer()

## Word Embedding

In [3]:
# corpus = datahelper.prepare_word_embedding_corpus(
#     '../data/text8', 
#     TOKEN_TYPE, 
#     REMOVE_STOPWORDS, 
#     TO_LOWER,
# )

# wordemb_model = Word2Vec(corpus, size=200, min_count=0, workers=4, iter=30, window=8)
# word_vectors = wordemb_model.wv

In [4]:
# word_vectors.save("../embedding/STCWiki/STCWiki_mincount0.model.bin")
# datahelper.set_word_vectors(word_vectors)

## Read Corpus & Prepare data for model

In [5]:
def get_data():
    trainX, trainND, trainDQ, train_turns, train_masks = datahelper.get_model_train_data(
        'train',
        TOKEN_TYPE, 
        REMOVE_STOPWORDS, 
        TO_LOWER,
        EMB,
    )

    devX, devND, devDQ, dev_turns, dev_masks = datahelper.get_model_train_data(
        'dev',
        TOKEN_TYPE, 
        REMOVE_STOPWORDS, 
        TO_LOWER,
        EMB,
    )

    testX, test_turns, test_masks = datahelper.get_model_test_data(
        TOKEN_TYPE, 
        REMOVE_STOPWORDS, 
        TO_LOWER,
        EMB,
    )

    testIDs = datahelper.testIDs
    trainDQA = [item['A'] for item in trainDQ]
    trainDQS = [item['S'] for item in trainDQ]
    trainDQE = [item['E'] for item in trainDQ]
    devDQA = [item['A'] for item in devDQ]
    devDQS = [item['S'] for item in devDQ]
    devDQE = [item['E'] for item in devDQ]

    dataND = [trainX, trainND, train_turns, train_masks, devX, devND, dev_turns, dev_masks, testX, test_turns, test_masks]
    dataDQA = [trainX, trainDQA, train_turns, devX, devDQA, dev_turns, testX, test_turns]
    dataDQS = [trainX, trainDQS, train_turns, devX, devDQS, dev_turns, testX, test_turns]
    dataDQE = [trainX, trainDQE, train_turns, devX, devDQE, dev_turns, testX, test_turns]

    return dataND, dataDQA, dataDQS, dataDQE, testX, test_turns, test_masks, testIDs

In [6]:
# dataND, dataDQA, dataDQS, dataDQE, testX, test_turns, test_masks, testIDs = get_data()

In [31]:
es = 3
fixed_paramsND  = {
    'epoch':100, 
    'early_stopping':es, 
    'batch_size':30,
    'lr':5e-4,
    'kp':1, 
    'hiddens':1024, 
    'Fsize':[2,2], 
    'gating':False, 
    'bn':True, 
    'method':ND.CNNRNN,
} 

fixed_paramsDQ = {
    'epoch':50, 
    'early_stopping':es, 
    'batch_size':40, 
    'kp':1, 
    'lr':0.001, 
    'hiddens':1024, 
    'Fsize':[2,2],
}

fixed_paramsDQA = fixed_paramsDQ.update({'scoretype':'DQA'})
fixed_paramsDQS = fixed_paramsDQ.update({'scoretype':'DQS'})
fixed_paramsDQE = fixed_paramsDQ.update({'scoretype':'DQE'})

In [32]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
def show_train_history(title, train, valid, earlystop=es):
    epoch = len(train)
    best = epoch-earlystop
    x = [i for i in range(1, epoch + 1)]
    plt.figure(figsize=(5,12))
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.rcParams.update({'font.size': 18})
    plt.plot(x, train, marker='o', linestyle='-', color='b')
    plt.plot(x, valid, marker='o', linestyle='-', color='r')
    plt.axvline(best, color='black')
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
    plt.show()

## Test ND

In [33]:
BEST_PATH = 'ResultPickle/'
# bestND = pickle.load(open(BEST_PATH + 'bestND.p', "rb"))
bestDQAs = pickle.load(open(BEST_PATH + 'memoryDQAs.p', "rb"))
bestDQSs = pickle.load(open(BEST_PATH + 'memoryDQSs.p', "rb"))
bestDQEs = pickle.load(open(BEST_PATH + 'memoryDQEs.p', "rb"))

In [34]:
corpus = datahelper.prepare_word_embedding_corpus(
    '../data/text8', 
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
)

In [36]:
size=100

for i in [15,20,25]:
    for ws in [5, 8]:
        for sg in [0, 1]:
            wordemb_model = Word2Vec(corpus, size=size, min_count=0, workers=4, iter=i, sg=sg, window=ws)
            word_vectors = wordemb_model.wv
            datahelper.set_word_vectors(word_vectors)
            dataND, dataDQA, dataDQS, dataDQE, testX, test_turns, test_masks, testIDs = get_data()

            e = True
            for fn in [[256, 512, 1024]]:
                for num_layers in [2]:
                    testname = 'iter{}_lr{}_sg{}_ws{}'.format(i, lr,sg,ws)
    #                 testname = 'emb{}_iter{}_Memory{}_GatingFalse_CNNRNN_{}stack_rnnLayer{}'.format(size, i, mr, len(fn), num_layers)
                    print(testname)
                    testND, train_losses, dev_losses = stctrain.start_trainND(
                        *dataND, 
                        **fixed_paramsND,
                        Fnum=fn, num_layers=num_layers, memory_rnn_type=None,
                        evaluate=e,
                    )

        #             show_train_history(testname, train_losses, dev_losses)
                    datahelper.pred_to_submission(testND, bestDQAs[0], bestDQSs[0], bestDQEs[0], test_turns, testIDs, filename='{}.json'.format(testname))

iter15_lr0.0005_sg0_ws5
x.shape (?, 7, 150, 100)
CNNRNN|14|False|True|2_2|1024|256_512_1024|1|0.0257|0.0958
iter15_lr0.0005_sg1_ws5
x.shape (?, 7, 150, 100)
CNNRNN|10|False|True|2_2|1024|256_512_1024|1|0.0285|0.1015
iter15_lr0.0005_sg0_ws8
x.shape (?, 7, 150, 100)
CNNRNN|14|False|True|2_2|1024|256_512_1024|1|0.0261|0.0967
iter15_lr0.0005_sg1_ws8
x.shape (?, 7, 150, 100)
CNNRNN|13|False|True|2_2|1024|256_512_1024|1|0.0262|0.0972
iter20_lr0.0005_sg0_ws5
x.shape (?, 7, 150, 100)
CNNRNN|8|False|True|2_2|1024|256_512_1024|1|0.0337|0.1084
iter20_lr0.0005_sg1_ws5
x.shape (?, 7, 150, 100)
CNNRNN|16|False|True|2_2|1024|256_512_1024|1|0.0249|0.0961
iter20_lr0.0005_sg0_ws8
x.shape (?, 7, 150, 100)
CNNRNN|12|False|True|2_2|1024|256_512_1024|1|0.0272|0.0996
iter20_lr0.0005_sg1_ws8
x.shape (?, 7, 150, 100)
CNNRNN|11|False|True|2_2|1024|256_512_1024|1|0.0275|0.1014
iter25_lr0.0005_sg0_ws5
x.shape (?, 7, 150, 100)
CNNRNN|9|False|True|2_2|1024|256_512_1024|1|0.0306|0.1024
iter25_lr0.0005_sg1_ws5
x.shap

In [12]:
# pickle.dump(testND, open('bestND190116.p', 'wb'))

In [13]:
stop

NameError: name 'stop' is not defined

## Test DQ

In [ ]:
memoryNDs = pickle.load(open('ResultPickle/memoryNDs.p', 'rb'))

In [ ]:
e = True
for rm in ['Bi-GRU', 'Bi-LSTM']:
    for l in [1, 2, 3]:
        for fn in [[1024], [512, 1024], [256, 512, 1024]]:
            testname = 'A-GatingTrue_{}stack_Memory{}_RNNLayer{}'.format(len(fn), rm, l)
            print(testname)
            bestDQA, train_lossesA, dev_lossesA = stctrain.start_trainDQ(
                *dataDQA, 
                **fixed_paramsDQA, 
                Fnum=fn, gating=True, bn=True, method=DQ.CNNCNN, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )

#             show_train_history(testname, train_lossesA, dev_lossesA)

            testname = 'E-GatingTrue_{}stack_Memory{}_RNNLayer{}'.format(len(fn), rm, l)
            print(testname)
            bestDQE, train_lossesE, dev_lossesE = stctrain.start_trainDQ(
                *dataDQE, 
                **fixed_paramsDQE, 
                Fnum=fn, gating=True, bn=True, method=DQ.CNNCNN, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )

#             show_train_history(testname, train_lossesE, dev_lossesE)

            testname = 'S-GatingTrue_{}stack_Memory{}_RNNLayer{}'.format(len(fn), rm, l)
            print(testname)
            bestDQS, train_lossesS, dev_lossesS = stctrain.start_trainDQ(
                *dataDQS, 
                **fixed_paramsDQS, 
                Fnum=fn, gating=True, bn=True, method=DQ.CNNCNN, memory_rnn_type=rm, num_layers=l,
                evaluate=e
            )

#             show_train_history(testname, train_lossesS, dev_lossesS)

            testname = 'GatingTrue_{}stack_Memory{}_RNNLayer{}'.format(len(fn), rm, l)
            datahelper.pred_to_submission(memoryNDs[0], bestDQA, bestDQS, bestDQE, test_turns, testIDs, filename='{}.json'.format(testname))
            print(testname, 'is saved')

In [ ]:
# pickle.dump(bestDQAs, open('memoryDQAs.p', 'wb'))
# pickle.dump(bestDQSs, open('memoryDQSs.p', 'wb'))
# pickle.dump(bestDQEs, open('memoryDQEs.p', 'wb'))

## Test NDF

In [ ]:
memoryND = pickle.load(open('ResultPickle/bestND190116.p', 'rb'))

In [ ]:
testNDmasked = [np.multiply(nd, mask) for nd, mask in zip(memoryND, test_masks)]

In [ ]:
dataDQA_NDF = [trainX, trainDQA, train_turns, trainND, devX, devDQA, dev_turns, devND, testX, test_turns, testNDmasked]
dataDQS_NDF = [trainX, trainDQS, train_turns, trainND, devX, devDQS, dev_turns, devND, testX, test_turns, testNDmasked]
dataDQE_NDF = [trainX, trainDQE, train_turns, trainND, devX, devDQE, dev_turns, devND, testX, test_turns, testNDmasked]

In [ ]:
method = DQNDF.CNNCNN
e = True
for rm in [None]:
    for l in [1]:
        for fn in [[512, 1024]]:
            testname = 'DQNDFA-GatingTrue_{}stack_Memory{}_{}{}'.format(len(fn), rm, l, str(method).split()[1])
            print(testname)
            bestDQNDFA, train_lossesA, dev_lossesA = stctrain.start_trainDQ_NDF(
                *dataDQA_NDF, 
                **fixed_paramsDQA, 
                Fnum=fn, gating=True, bn=True, method=method, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )

#             show_train_history(testname, train_lossesA, dev_lossesA)

            testname = 'DQNDFE-GatingTrue_{}stack_Memory{}_{}{}'.format(len(fn), rm, l, str(method).split()[1])
            print(testname)
            bestDQNDFE, train_lossesE, dev_lossesE = stctrain.start_trainDQ_NDF(
                *dataDQE_NDF, 
                **fixed_paramsDQE, 
                Fnum=fn, gating=True, bn=True, method=method, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )

#             show_train_history(testname, train_lossesE, dev_lossesE)

            testname = 'DQNDFS-GatingTrue_{}stack_Memory{}_{}{}'.format(len(fn), rm, l, str(method).split()[1])
            print(testname)
            bestDQNDFS, train_lossesS, dev_lossesS = stctrain.start_trainDQ_NDF(
                *dataDQS_NDF, 
                **fixed_paramsDQS, 
                Fnum=fn, gating=True, bn=True, method=method, memory_rnn_type=rm, num_layers=l,
                evaluate=e
            )

#             show_train_history(testname, train_lossesS, dev_lossesS)

            testname = 'DQNDF-GatingTrue_{}stack_Memory{}_{}{}'.format(len(fn), rm, l, str(method).split()[1])
            datahelper.pred_to_submission(memoryND, bestDQNDFA, bestDQNDFS, bestDQNDFE, test_turns, testIDs, filename='{}.json'.format(testname))
            print(testname, 'is saved')

In [ ]:
# pickle.dump(bestDQANDFs, open('memoryDQANDFs.p', 'wb'))
# pickle.dump(bestDQSNDFs, open('memoryDQSNDFs.p', 'wb'))
# pickle.dump(bestDQENDFs, open('memoryDQENDFs.p', 'wb'))

In [ ]:
import numpy as np
import tensorflow as tf

# Data settings.
num_examples = 10
num_words = 20
num_features = 100
num_tags = 5

# Random features.
x = np.random.rand(num_examples, num_words, num_features).astype(np.float32)

# Random tag indices representing the gold sequence.
y = np.random.randint(num_tags, size=[num_examples, num_words]).astype(np.int32)

print(y)

# All sequences in this example have the same length, but they can be variable in a real model.
sequence_lengths = np.full(num_examples, num_words - 1, dtype=np.int32)
print(sequence_lengths)

# Train and evaluate the model.
with tf.Graph().as_default():
  with tf.Session() as session:
    # Add the data to the TensorFlow graph.
    x_t = tf.constant(x)
    y_t = tf.constant(y)
    sequence_lengths_t = tf.constant(sequence_lengths)

    # Compute unary scores from a linear layer.
    weights = tf.get_variable("weights", [num_features, num_tags])
    matricized_x_t = tf.reshape(x_t, [-1, num_features])
    matricized_unary_scores = tf.matmul(matricized_x_t, weights)
    unary_scores = tf.reshape(matricized_unary_scores,
                              [num_examples, num_words, num_tags])

    # Compute the log-likelihood of the gold sequences and keep the transition
    # params for inference at test time.
    print(unary_scores.shape)
    print(y_t.shape)
    print(sequence_lengths_t.shape)
    log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
        unary_scores, y_t, sequence_lengths_t)

    # Add a training op to tune the parameters.
    loss = tf.reduce_mean(-log_likelihood)
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # Train for a fixed number of iterations.
    session.run(tf.global_variables_initializer())
    for i in range(1000):
      tf_unary_scores, tf_transition_params, _ = session.run(
          [unary_scores, transition_params, train_op])
      if i % 100 == 0:
        correct_labels = 0
        total_labels = 0
        for tf_unary_scores_, y_, sequence_length_ in zip(tf_unary_scores, y,
                                                          sequence_lengths):
          # Remove padding from the scores and tag sequence.
          tf_unary_scores_ = tf_unary_scores_[:sequence_length_]
          y_ = y_[:sequence_length_]

          # Compute the highest scoring sequence.
          viterbi_sequence, _ = tf.contrib.crf.viterbi_decode(
              tf_unary_scores_, tf_transition_params)

          # Evaluate word-level accuracy.
          correct_labels += np.sum(np.equal(viterbi_sequence, y_))
          total_labels += sequence_length_
        accuracy = 100.0 * correct_labels / float(total_labels)
        print("Accuracy: %.2f%%" % accuracy)